In [1]:
import pandas as pd
import datetime

In [2]:
import pandas

In [3]:
pd.options.display.max_columns=500
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

In [4]:
result = []
render = False

In [5]:
from dqn import Qnet
from torchsummary import summary

summary(Qnet(), (3,84,84))

D:\develop\tools\anaconda3\envs\rl-learn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x5408 and 12800x4096)

In [ ]:
def print_result(df):
    
    df = df.copy()
    
    for i in range(10, len(df), 10):
        print(f"episode {i} average reward: {df[i-10:i].mean()['reward']} max reward: {df[i-10:i].max()['reward']}")

### Global Varaiable & Property

In [ ]:
# global variable


agent = None
target_epsilon = None
savefile = None


# global property

render = False
show_action = False

### CODE

In [ ]:
import gym

import random
import collections

import time
import threading

from dqn import *


def main():

    parameter = HyperParameter(
        batch_size=64,
        buffer_limit=50000,
        gamma=0.98,
        learning_rate=0.001
    )
    
    global agent
    global epsilon
    global render
    
    env = Environment()
    agent = DQNAgent(param=parameter, path=savefile)

    print_interval = 20
    score = 0.0

    for n_epi in range(100000):
        epsilon = max(0.01, 0.2 - 0.1 * (n_epi / 10000))
        
        if target_epsilon:
            epsilon = target_epsilon
        
        state = env.reset()
        done = False
        
        action_list = []
        episode_reward = 0
        
        while not done:
            action = agent.predict(state, epsilon)
            state_prime, reward, done, info = agent.step(env, state, action)

            state = state_prime
            
            score += reward   
            episode_reward += reward
            
            if render:
                env.render()
                import time
                time.sleep(0.01)
            
            if show_action:
                action_list.append(action)
        
        
        if show_action:
            print(action_list)
            action_list = []
        
        result.append({
            'reward': episode_reward,
            'epsilon': epsilon
        })
        
        if n_epi % print_interval == 0 and n_epi != 0:
            agent.update_network()

            print("n_episode :{}, score : {:.1f}, eps : {:.1f}%".format(
                n_epi, score / print_interval, epsilon * 100))
            score = 0.0

threading.Thread(target=main).start()            
#main()

In [ ]:
df = pd.DataFrame(result)
df

In [ ]:
print_result(df)

In [ ]:
render = False

In [ ]:
raise Exception

In [ ]:
agent.save('20220518121700.pt')

[20220517 084300]
유령 먹는 점수가 eat 점수에 비해 커가지고 agent가 야비하게 아이템을 먹고 쳐 기다린다... 보상점수를 좀 줄여야 할 듯 하다.
결국엔 전체적인 게임 흐름이 아니라 독립되어 한 상태에서 보상 값읅 구하는거기때문에 유령 먹는점수를 하향하는게 매우 중요하다
왜 계속 구석에 가있나 햇네 ㅋㅋㅋ


ghost_reward 변수를 새롭게 정의후 200에서 0으로 수정 유령은 필수로 먹어야하는 요소가 아니며 살아남기 위한 요소라거 정의하도록 한다.

[20220518 083000] 한 목숨 당 death 제어하는거 제외 이유는 이전에 높은 점수를 얻었는데도 불구하고
해당 구조로 인해 쿠키를 많이 먹지 못한 시나리오에 비해 우선도가 떨어질수도 있기 때문\

빨리 죽을 수 있는 문제는 엡실론 값을 주어 죽는게 아닌 다른 방법을 취해 쿠키를 먹었을때 보상을 얻어 점수가 쌓이게 되닌깐 결국엔 그방향ㅇ
으로 갈것이라 예상됨

[20220518 121600] 왜 계속 팩맨이 구석탱이로 가나 생각해보니 데스에 대한 마이너스 값을 너무 크게 해서 팩맨이 소녀감성을 갖게되서
발생된 일이 아닌가 싶음... 타켓은 다음 상태를 의미하기 때문에 전체적인 관점에선 아닐 수 있으나 단일 시점에선 그럴 가능성이 있음 사망 값을 좀 낮추어볼까...? (Q-learning을 사용한다면 전체적인 시점이 아닌 짧은 시점에서 찾아보는 것은 어떨까 애초에 td(시간차 방식)으로 측정하므로

함수 근사치로 구한다는 것을 명심하자